In [149]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

import warnings
warnings.filterwarnings("ignore")

In [150]:
train_data=pd.read_csv("tweets_train.csv")
train_data.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source,is_retweet,clean_text,Sentiment
0,Romy 👑,Bolton - England,"Tables turn, bridges burn, you live and learn.",2009-06-15 09:00:39,525,896,3854,False,2020-09-19 15:19:32,['TheSocialDilemma'],Twitter for Android,False,TheSocialDilemma is an eye opener isn t it ple...,Neutral
1,TLynn Peterson,"Black Canyon City, Arizona",Acquired disability ♿ after an accident. Livin...,2013-05-29 00:17:46,5045,5374,48152,False,2020-09-13 00:31:46,['TheSocialDilemma'],Twitter for Android,False,TheSocialDilemma If we don t agree on what is ...,Positive
2,Rutger Kosters,Netherlands,Cloud Solution Architect @NetApp | VCDX #209 |...,2014-10-07 09:34:10,574,251,1070,False,2020-09-09 21:10:33,['TheSocialDilemma'],Twitter for Android,False,Watching TheSocialDilemma scary to see social ...,Negative
3,IDFWU🗯🇿🇼🇿🇦,"East London, South Africa","YOU HAVE OPTIONS, YOU CAN’T JUST TAKE WHAT LIF...",2012-08-22 11:09:06,706,618,1320,False,2020-09-11 04:33:08,NaN,Twitter for Android,False,You check your social media before you pee in ...,Positive
4,Treebel,NaN,NaN,2010-09-25 07:25:31,9,13,55,False,2020-09-17 16:27:38,['thesocialdilemma'],Twitter for iPhone,False,watch thesocialdilemma and see what s actually...,Negative


In [151]:
test_data=pd.read_csv("tweets_test.csv")
test_data.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source,is_retweet,clean_text
0,Doug Webb,Probably Europe,"I develop methods for effective cooperation, w...",2015-12-12 15:22:58,157,228,496,False,2020-09-15 22:53:36,"['theSocialDilemma', 'joinMastodon']",Twitter Web App,False,Watch theSocialDilemma then joinMastodon
1,Dan Reynish,"Medicine Hat, Alberta",News Anchor at Noon and 5 pm on CHAT TV. Canad...,2011-09-10 12:17:10,1722,2500,13912,False,2020-09-12 04:43:03,['WeTheNorth'],Twitter Web App,False,With the 2019 2020 NBA season officially over ...
2,🌤,NaN,NaN,2011-06-01 02:23:57,880,585,205,False,2020-09-19 07:06:53,['TheSocialDilemma'],Twitter for iPhone,False,if you want to really know about people go tal...
3,“IT IS WHAT IT IS.”🇰🇪🇳🇿🇺🇸,WA,Heliophile🌞☀️• #Rugby • Wildlifer • #HIV • #So...,2009-04-17 13:35:02,1143,2692,11999,False,2020-09-12 18:28:18,"['TheSocialDilemma', 'Netflix']",Twitter for iPhone,False,Have you watched TheSocialDilemma on Netflix
4,Mahima Sood,NaN,Amy Dunne X Holly Golightly || Data Scientist ...,2018-01-26 21:57:22,18,65,263,False,2020-09-12 18:48:53,['TheSocialDilemma'],Twitter for iPhone,False,Highly recommend TheSocialDilemma on


In [152]:
train_data.shape

(17000, 14)

In [153]:
test_data.shape

(3068, 13)

In [154]:
train_data.isnull().sum()

user_name              1
user_location       3607
user_description    1162
user_created           0
user_followers         0
user_friends           0
user_favourites        0
user_verified          0
date                   0
hashtags            3649
source                 0
is_retweet             0
clean_text             8
Sentiment              0
dtype: int64

In [155]:
test_data.isnull().sum()

user_name             0
user_location       601
user_description    221
user_created          0
user_followers        0
user_friends          0
user_favourites       0
user_verified         0
date                  0
hashtags            648
source                0
is_retweet            0
clean_text            1
dtype: int64

In [156]:
train_data.fillna("", inplace=True)
test_data.fillna("", inplace=True)

In [157]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['clean_text'])
sequences = tokenizer.texts_to_sequences(train_data['clean_text'])
max_sequence_length = max([len(x) for x in sequences])
word_index = tokenizer.word_index

In [158]:
data = pad_sequences(sequences, maxlen=max_sequence_length)

In [159]:
target = pd.get_dummies(train_data['Sentiment'])

In [160]:
X_train, X_val, y_train, y_val = train_test_split(data, target, test_size=0.2, random_state=42)

In [161]:
model = Sequential()

In [162]:
model.add(Embedding(len(word_index) + 1, 100, input_length=max_sequence_length))

In [163]:
model.add(SimpleRNN(128))

In [164]:
model.add(Dense(3, activation='softmax'))

In [165]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [166]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=64)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
211/213 [============================>.] - ETA: 0s - loss: 0.7799 - accuracy: 0.6628WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001C4A9BD8318> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`

In [168]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy}')

107/107 [==============================] - 0s 3ms/step - loss: 0.7748 - accuracy: 0.8585
Validation Accuracy: 0.8585293889045715


In [169]:
test_sequences = tokenizer.texts_to_sequences(test_data['clean_text'])
test_data_padded = pad_sequences(test_sequences, maxlen=max_sequence_length)

In [170]:
predictions = model.predict(test_data_padded)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
96/96 [==============================] - 0s 3ms/step


In [171]:
sentiment_labels = ['Negative', 'Neutral', 'Positive']
predicted_labels = [sentiment_labels[np.argmax(pred)] for pred in predictions]

In [172]:
test_data['Predicted_Sentiment'] = predicted_labels

In [173]:
test_data.to_csv("test_data_with_predictions.csv", index=False)

In [174]:
df=pd.read_csv("test_data_with_predictions.csv")
df

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source,is_retweet,clean_text,Predicted_Sentiment
0,Doug Webb,Probably Europe,"I develop methods for effective cooperation, w...",2015-12-12 15:22:58,157,228,496,False,2020-09-15 22:53:36,"['theSocialDilemma', 'joinMastodon']",Twitter Web App,False,Watch theSocialDilemma then joinMastodon,Neutral
1,Dan Reynish,"Medicine Hat, Alberta",News Anchor at Noon and 5 pm on CHAT TV. Canad...,2011-09-10 12:17:10,1722,2500,13912,False,2020-09-12 04:43:03,['WeTheNorth'],Twitter Web App,False,With the 2019 2020 NBA season officially over ...,Neutral
2,🌤,NaN,NaN,2011-06-01 02:23:57,880,585,205,False,2020-09-19 07:06:53,['TheSocialDilemma'],Twitter for iPhone,False,if you want to really know about people go tal...,Positive
3,“IT IS WHAT IT IS.”🇰🇪🇳🇿🇺🇸,WA,Heliophile🌞☀️• #Rugby • Wildlifer • #HIV • #So...,2009-04-17 13:35:02,1143,2692,11999,False,2020-09-12 18:28:18,"['TheSocialDilemma', 'Netflix']",Twitter for iPhone,False,Have you watched TheSocialDilemma on Netflix,Neutral
4,Mahima Sood,NaN,Amy Dunne X Holly Golightly || Data Scientist ...,2018-01-26 21:57:22,18,65,263,False,2020-09-12 18:48:53,['TheSocialDilemma'],Twitter for iPhone,False,Highly recommend TheSocialDilemma on,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3063,𝘔𝘶𝘳𝘪;🦋,𝘶𝘯𝘥𝘦𝘳 𝘵𝘩𝘦 𝘱𝘢𝘭𝘮𝘴🌴,𝘸𝘪𝘧𝘦 // 𝘤𝘢𝘵 𝘮𝘰𝘮 // 𝘱𝘴𝘺𝘤𝘩𝘰𝘭𝘰𝘨𝘪𝘴𝘵; 🧠 𝘣𝘦𝘢𝘤𝘩𝘣𝘶𝘮;🌴,2009-07-29 03:18:16,942,357,9917,False,2020-09-14 12:27:35,['TheSocialDilemma'],Twitter for iPhone,False,Everybody should watch TheSocialDilemma,Neutral
3064,Didi Crawford,"Birmingham, England",Traveller | Blogger | Currently growing a huma...,2011-12-06 09:24:20,468,1095,3440,False,2020-09-22 19:25:48,['TheSocialDilemma'],Twitter for iPhone,False,Enjoyed TheSocialDilemma Then read this,Neutral
3065,Ritwik Arora,NaN,"Spent the last 30 minutes writing this bio, ch...",2020-10-13 01:12:06,10,137,106,False,2020-10-13 02:41:26,['TheSocialDilemma'],Twitter for Android,False,Couldn t stop thinking about docu TheSocialDil...,Negative
3066,Nishant Mody,Mumbai,"I love music, dogs, nature, politics & marketi...",2012-12-30 04:58:15,174,541,5482,False,2020-09-10 17:19:44,['TheSocialDilemma'],Twitter for iPhone,False,This was the best one by on TheSocialDilemma on,Positive
